# Day Shifts

In [1]:
from src.read_csv import parse_csv
from src.schedule import Person, Schedule

import os
import pandas as pd
import copy
import statistics
from itertools import combinations

In [2]:
# Minimum number of people that need to be assigned during day shifts
min_people_day = 1

In [3]:
availabilities = [parse_csv(os.path.join('data', file)) for file in os.listdir('data') if file.endswith('.csv')]

print(f'Number of availabilities: {len(availabilities)}')
print([avail.columns[0] for avail in availabilities])

Number of availabilities: 2
['Michael Bryant', 'Barack Obama']


In [4]:
# Sum all the availabilities element-wise, except ignoring if the value is non-numerical
# Therefore, total_avail will contain the total number of people available for each shift

total_avail = None

for df in availabilities:

    df2 = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    if total_avail is None:
        total_avail = df2
    else:
        total_avail += df2

# Add back the times (first column) to the total availability
total_avail = pd.concat([df.iloc[:, 0], total_avail], axis=1)
total_avail.columns = ['Total Available', *total_avail.columns[1:]]

# Remove the last row which is night shift
total_avail = total_avail.iloc[:-1, :]

In [5]:
# Now for the actual scheduling

display(total_avail)


,Total Available,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,7:30 - 8:00 AM,2,2,2,2,2,2,2
1,8:00 - 8:30 AM,2,2,2,2,2,2,1
2,8:30 - 9:00 AM,2,2,2,2,2,2,2
3,9:00 - 9:30 AM,2,2,2,2,2,2,2
4,9:30 - 10:00 AM,2,2,2,2,2,2,2
5,10:00 - 10:30 AM,2,2,2,2,2,2,2
6,10:30 - 11:00 AM,1,2,2,2,2,2,2
7,11:00 - 11:30 AM,2,2,2,2,2,2,2
8,11:30 - 12:00 PM,2,2,2,2,2,2,1
9,12:00 -12:30 PM,2,2,2,2,2,2,1


In [6]:
people = []

for p in availabilities:
    name = p.columns[0]
    avail_mask = p.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values
    avail_mask = avail_mask[:, 1:]
    people.append(Person(name, avail_mask, time_slots=p.shape[0]-1))

night_schedule = Schedule(people, min_people_per_slot=min_people_day)
night_schedule.find_schedules(display)